In [ ]:
# Importation des bibliothèques nécessaires
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Chargement de la série temporelle des futures WTI
# Assurez-vous que le fichier contient des colonnes 'Timestamp' et 'Price'
data = pd.read_csv("wti_futures_data.csv", parse_dates=['Timestamp'])

# Tri des données par date et heure si nécessaire
data = data.sort_values('Timestamp').reset_index(drop=True)

# Calcul de la volatilité intrajournalière (écart-type des rendements) sur des intervalles de 30 minutes
data.set_index('Timestamp', inplace=True)
data['Return'] = data['Price'].pct_change()  # Calcul des rendements

# Resample la série temporelle par tranches de 30 minutes
volatility_30min = data['Return'].rolling('30min').std() * np.sqrt(30)  # Annualiser si besoin

# Détection des anomalies - seuil simple basé sur une déviation standard élevée (3 sigma, par exemple)
threshold = volatility_30min.mean() + 3 * volatility_30min.std()  # Ajustable en fonction de la tolérance
anomalies = volatility_30min[volatility_30min > threshold]

# Visualisation de la volatilité et des anomalies
plt.figure(figsize=(12, 6))
plt.plot(volatility_30min, label="30-Min Volatility")
plt.scatter(anomalies.index, anomalies, color='red', label="Volatility Spikes (Anomalies)", marker='o')
plt.xlabel("Time")
plt.ylabel("Volatility")
plt.title("30-Min Volatility with Identified Spikes")
plt.legend()
plt.show()
